**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

You've built up your SQL skills enough that the remaining hands-on exercises will use different datasets than you see in the explanations. If you need to familiarize with a new dataset, you can run a couple of `SELECT` queries to extract and review the data you need. 

The next exercises are also more challenging than what you've done so far. Don't worry, you are ready for it.

Run the code in the following cell to get everything set up:

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex4 import *

# import package with helper functions 
import bq_helper


# create a helper object for this dataset
education_data = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                          dataset_name="world_bank_intl_education")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


Then write the code to answer the questions below.

In [19]:
from google.cloud import bigquery as bigq

bq = bigq.Client()
print("imported: ", bq)
active_project = "bigquery-public-data"
dataset_name = "world_bank_intl_education"

data_ref = bq.dataset("world_bank_intl_education", project = "bigquery-public-data")

# data_ref = bq.dataset(dataset_name, project = active_project)

data = bq.get_dataset(data_ref)

tables = list(bq.list_tables(data))
# print(tables.table_id)

table_ref = data_ref.table('international_education')
edutable = bq.get_table(table_ref)

edutable.schema


# education_data

Using Kaggle's public dataset BigQuery integration.
imported:  <kaggle_gcp.PublicBigqueryClient object at 0x7f892c941630>


[SchemaField('country_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('country_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', '', ()),
 SchemaField('year', 'INTEGER', 'NULLABLE', '', ())]

In [9]:
data.information_schema

AttributeError: 'Dataset' object has no attribute 'information_schema'

# Questions

## 1) Which countries spend the largest fraction of GDP on education?

The World Bank has made tons of interesting education data available through BigQuery. Run the following cell to see the first few rows of the table.

In [20]:
education_data.head('international_education')

,country_name,country_code,indicator_name,indicator_code,value,year
0,East Asia & Pacific (excluding high income),EAP,"Labor force, female (% of total labor force)",SL.TLF.TOTL.FE.ZS,4.326169e+01,2016
1,Europe & Central Asia (excluding high income),ECA,"Population, ages 0-14, female",SP.POP.0014.FE.IN,4.153863e+07,2016
2,Europe & Central Asia (excluding high income),ECA,"Population, ages 0-14, total",SP.POP.0014.TO,8.540403e+07,2016
3,Heavily indebted poor countries (HIPC),HPC,"Population, ages 0-14, male",SP.POP.0014.MA.IN,1.629388e+08,2016
4,High income,HIC,GDP per capita (current US$),NY.GDP.PCAP.CD,4.080358e+04,2016


The indicator code describes what type of data is shown in a given row.  

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`. The name says this is `Government expenditure on education as % of GDP (%)`.

Write a query to get a dataframe returning **average value of these rows for each country in the dataset between the years 2010-2017** (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregation for an average is **AVG**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different database):
```
# query to find how many accidents happen each day of the week
query = """SELECT COUNT(consecutive_number) num_accidents, 
                  EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
            GROUP BY EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
            ORDER BY COUNT(consecutive_number) DESC
        """
accidents_by_day = accidents.query_to_pandas_safe(query)
```

In [35]:
# Your Code Here

# https://www.blendo.co/blog/access-data-google-bigquery-python-r/
# 
# query_job = client.query("""
#   SELECT *
#   FROM dataset.my_table
#   LIMIT 1000""", job_config=job_config)
# results = query_job.result()  # Waits for job to complete.

safe_config = bigq.QueryJobConfig(maximum_bytes_billed=1e9)



country_spend_pct_query = """
SELECT country_name, AVG(value) AS avg_ed_spending_pct
FROM `bigquery-public-data.world_bank_intl_education.international_education`
WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year <= 2017 AND year >= 2010
GROUP BY country_name
ORDER BY `avg_ed_spending_pct` DESC
"""

quer = bq.query(country_spend_pct_query, job_config=safe_config)
q = quer.result()
q = q.to_dataframe
print("Mine: ", q)
print("==============")

country_spending_results = education_data.query_to_pandas_safe(country_spend_pct_query)

print(country_spending_results.head())
q_1.check()

Mine:  <google.cloud.bigquery.table.RowIterator object at 0x7f892c6bc9b0>
            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [39]:
q = quer.result()

q = q.to_dataframe()
print(q)

                 country_name  avg_ed_spending_pct
0                        Cuba            12.837270
1       Micronesia, Fed. Sts.            12.467750
2             Solomon Islands            10.001080
3                     Moldova             8.372153
4                     Namibia             8.349610
5                     Denmark             8.274300
6                 Timor-Leste             7.975114
7                     Iceland             7.480210
8                      Sweden             7.233168
9                       Malta             7.134535
10                     Norway             6.986035
11                    Finland             6.909100
12                    Bolivia             6.892202
13                   Zimbabwe             6.817810
14                 Costa Rica             6.788503
15                      Ghana             6.775490
16                New Zealand             6.752527
17                    Senegal             6.637846
18                  Swaziland  

In [ ]:
# q_1.hint()
# q_1.solution()

## 2) Identifying Interesting Codes to Explore
The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write the query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- One row in your results per code
- The columns in your results should be called `indicator_code`, `indicator_name` and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT** aggregation). This in turn requires you to include both in your **GROUP BY** clause.
- Order results from most frequent to least frequent.

In [ ]:
# Your Code Here
code_count_query = """____"""

code_count_results = education_data.query_to_pandas_safe(code_count_query)

print(code_count_results.head())
q_2.check()

In [ ]:
q_2.hint()
q_2.solution()

---
# Keep Going
**[Click here](https://www.kaggle.com/dansbecker/as-with)** to learn how *WITH-AS* clauses  can clean up your code and help you construct more complex queries.


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

